In [1]:
!nvidia-smi

Sun Aug  4 09:36:00 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%pip install transformers==4.32 \
             tensorflow \
             datasets \
             loralib \
             peft -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
kaggle-environments 1.14.15 requires transformers>=4.33.1, but you have transformers 4.32.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import AutoModelForSeq2SeqLM
model_name = "facebook/nllb-200-distilled-600M"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
from transformers import NllbTokenizer
s_lang="hin_Deva"
t_lang="kan_Deva"
tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M', src_lang=s_lang, tgt_lang=t_lang)

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

## **Loading and preparing the Dataset: Kinori**

In [7]:
h_path= '/kaggle/input/ft-mt-llm-data/Parallel_data_Hi.txt'
k_path= '/kaggle/input/ft-mt-llm-data/Parallel_data_KP.txt'

def read_and_filter(file_path):
    sentences= []
    with open(file_path, 'r', encoding= 'utf-8') as file:
        for line in file:
            stripped_line= line.strip()
            if stripped_line:
                sentences.append(line)
                
    return sentences

h_sen= read_and_filter(h_path)
k_sen= read_and_filter(k_path)

if len(h_sen) != len(k_sen):
    raise ValueError("The number of sentences do not match!")
else:
    print("The number of sentences match!")

The number of sentences match!


In [8]:
print(f"Total number of sentences: {len(h_sen)}")

Total number of sentences: 20307


In [9]:
for i in range(5):
    print(f"Hindi: {h_sen[i]}")
    print(f"Kinori: {k_sen[i]}")
    print("--"*40)

Hindi: हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ लोग विदक जाते हैं।

Kinori: हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक मानुष सौर्मींदैसा।

--------------------------------------------------------------------------------
Hindi: अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों को कोई तकलीफ हो तो भी समझ आती है ।

Kinori: कियै हौयौ बातै कौरि ओरिक प्रदेशाए मानुषौलै किछ़ तकलीफ हुँदेन ताबिह सौमौज़ आछ़दौँस।

--------------------------------------------------------------------------------
Hindi: किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की के मन की संकीर्णता समझ नहीं आती ।

Kinori: पर आपनैए प्रदेशाई काद रिह ताईंयै मानुषौ रोह मौनै कैह मारे बौरताव सौमौज़ नाछ़दौस।

--------------------------------------------------------------------------------
Hindi: पता नहीं ऐसा कर के क्या हासिल होगा ।

Kinori: ज़ानि हैयौ कौरिंदै कयौ पौरेस।

--------------------------------------------------------------------------------
Hindi: अगर हम अपनी माता (हिमाचली भाषा) के नहीं हो सकते तो विमाता के कहाँ होगे  ।

Kin

In [10]:
dataset= list(zip(h_sen, k_sen))

import csv
import pandas as pd

file_name= "hi-ki-translations.csv"

with open(file_name, 'w', encoding= 'utf-8') as file:
    writer= csv.writer(file)
    writer.writerow(['Hindi', 'Kinori'])
    
    for Hindi, Kinori in dataset:
        writer.writerow([Hindi, Kinori])

In [11]:
dataframe= pd.read_csv("/kaggle/working/hi-ki-translations.csv")
dataframe

,Hindi,Kinori
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ...,हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक ...
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों क...,कियै हौयौ बातै कौरि ओरिक प्रदेशाए मानुषौलै किछ...
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की...,पर आपनैए प्रदेशाई काद रिह ताईंयै मानुषौ रोह मौ...
3,पता नहीं ऐसा कर के क्या हासिल होगा ।\n,ज़ानि हैयौ कौरिंदै कयौ पौरेस।\n
4,अगर हम अपनी माता (हिमाचली भाषा) के नहीं हो सकत...,कियै तामोरि आपनिये यो (हिमाचली बोली) रै निफिरि...
...,...,...
20302,जनाब ₹3 तो कंडक्टर को दे दिए पर 20 पैसे खुले ह...,जनाब त्रोन रुपया तह कंडक्टर लाह देई देह तेबिह ...
20303,पहले तो वह अस्पताल के लिए तैयार नहीं हुआ पर मु...,पोईले ताह सोह अस्‍पताल रोह ताई तैयार निफरिह ते...
20304,उसने मुट्ठी में गिन कर रखे हुए पैसे मेरे सामने...,तिनिये मुठोह केह गोनाई जागुन रुपया लाह मेरोह स...
20305,कई दिन गुजर गए मेरी चिट्ठी का भोलू राम पर क्य...,मेरे पास कोई आया गया ही नहीं यूनिवर्सिटी के इम...


> **Repeating to get more data:**

In [12]:
repeat_times = 3
augmented_dataframe = pd.concat([dataframe] * repeat_times)
augmented_dataframe = augmented_dataframe.reset_index(drop=True)
augmented_dataframe.to_csv("/kaggle/working/augmented_hi-ki-translations.csv")
augmented_dataframe

,Hindi,Kinori
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ...,हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक ...
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों क...,कियै हौयौ बातै कौरि ओरिक प्रदेशाए मानुषौलै किछ...
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की...,पर आपनैए प्रदेशाई काद रिह ताईंयै मानुषौ रोह मौ...
3,पता नहीं ऐसा कर के क्या हासिल होगा ।\n,ज़ानि हैयौ कौरिंदै कयौ पौरेस।\n
4,अगर हम अपनी माता (हिमाचली भाषा) के नहीं हो सकत...,कियै तामोरि आपनिये यो (हिमाचली बोली) रै निफिरि...
...,...,...
60916,जनाब ₹3 तो कंडक्टर को दे दिए पर 20 पैसे खुले ह...,जनाब त्रोन रुपया तह कंडक्टर लाह देई देह तेबिह ...
60917,पहले तो वह अस्पताल के लिए तैयार नहीं हुआ पर मु...,पोईले ताह सोह अस्‍पताल रोह ताई तैयार निफरिह ते...
60918,उसने मुट्ठी में गिन कर रखे हुए पैसे मेरे सामने...,तिनिये मुठोह केह गोनाई जागुन रुपया लाह मेरोह स...
60919,कई दिन गुजर गए मेरी चिट्ठी का भोलू राम पर क्य...,मेरे पास कोई आया गया ही नहीं यूनिवर्सिटी के इम...


In [13]:
from sklearn.model_selection import train_test_split
train_val_df, test_dataset= train_test_split(augmented_dataframe, test_size=0.1, random_state=40)
train_dataset, evaluation_dataset= train_test_split(train_val_df, test_size=0.1, random_state=40)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (49345, 2)
Validation dataset shape:  (5483, 2)
Testing dataset shape:  (6093, 2)


> **Converting pandas dataframe to pyarrow**

In [14]:
from datasets import Dataset

train_dataset= Dataset.from_pandas(train_dataset)
val_dataset= Dataset.from_pandas(evaluation_dataset)
test_dataset= Dataset.from_pandas(test_dataset)

train_dataset= train_dataset.remove_columns("__index_level_0__")
val_dataset= val_dataset.remove_columns("__index_level_0__")
test_dataset= test_dataset.remove_columns("__index_level_0__")

In [15]:
import datasets
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [16]:
main_dataset

DatasetDict({
    train: Dataset({
        features: ['Hindi', 'Kinori'],
        num_rows: 49345
    })
    validation: Dataset({
        features: ['Hindi', 'Kinori'],
        num_rows: 5483
    })
    test: Dataset({
        features: ['Hindi', 'Kinori'],
        num_rows: 6093
    })
})

## **Tokenizing and getting ready for Training: Kinori**

> **Checking the number of unknown tokens(UNK) in the dataset:**

In [17]:
from tqdm.auto import tqdm, trange
import random

texts_with_unk = [
    text for text in tqdm(dataframe.Hindi) 
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))

  0%|          | 0/20307 [00:00<?, ?it/s]

156


In [18]:
texts_with_unk = [
    text for text in tqdm(dataframe.Kinori) 
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))

  0%|          | 0/20307 [00:00<?, ?it/s]

364


The number of unknown tokens is fairly low therefore, putting off expanding the tokenizer vocabulary. 

>  **Adding a new language code to the tokenizer:**

In [19]:
def fix_tokenizer(tokenizer, new_lang='kan_Deva'):

    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang

    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}
    
    return tokenizer

tokenizer= fix_tokenizer(tokenizer)

In [20]:
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(256205, 1024)

In [21]:
added_token_id = tokenizer.convert_tokens_to_ids('kan_Deva')
similar_lang_id = tokenizer.convert_tokens_to_ids('hin_Deva')
embeds = model.model.shared.weight.data
# moving the embedding for "mask" to its new position
embeds[added_token_id+1] =embeds[added_token_id]
# initializing new language token with a token of a similar language
embeds[added_token_id] = embeds[similar_lang_id]

> **Tokenizing the main_dataset**

In [22]:
max_length= 128

def preprocess_function(examples):
    inputs= [ex for ex in examples["Hindi"]]
    targets= [ex for ex in examples["Kinori"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [23]:
tokenized_datasets = main_dataset.map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/49345 [00:00<?, ? examples/s]

Map:   0%|          | 0/5483 [00:00<?, ? examples/s]

Map:   0%|          | 0/6093 [00:00<?, ? examples/s]

In [25]:
tokenized_datasets= tokenized_datasets.remove_columns(['Hindi', 'Kinori'])

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 49345
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5483
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6093
    })
})

> **Data Collation**

In [27]:
%pip install torch -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [28]:
from transformers import DataCollatorForSeq2Seq
#generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt", pad_to_multiple_of=128)

#Pytorch
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

2024-08-04 06:11:24.185311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 06:11:24.185422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 06:11:24.320989: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
"""
# Since nllb doesn't support tensorlfow therefore going with pytorch
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids","attention_mask"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=generation_data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids","attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)
"""

'\n# Since nllb doesn\'t support tensorlfow therefore going with pytorch\ntf_train_dataset = tokenized_datasets["train"].to_tf_dataset(\n    columns=["input_ids","attention_mask"],\n    label_cols=["labels"],\n    shuffle=True,\n    collate_fn=generation_data_collator,\n    batch_size=8,\n)\n\ntf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(\n    columns=["input_ids","attention_mask"],\n    label_cols=["labels"],\n    shuffle=False,\n    collate_fn=data_collator,\n    batch_size=8,\n)\n'

In [33]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

## **Training the Model: Kinori**

In [12]:
%pip install sacrebleu \
             evaluate \
             accelerate -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [13]:
import evaluate
metric = evaluate.load("sacrebleu")

2024-08-04 11:16:50.099858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 11:16:50.099990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 11:16:50.277126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [34]:
"""
# Tensorflow not supported
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3

num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)
model.compile(optimizer=opt)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")
"""

'\n# Tensorflow not supported\nfrom tensorflow.keras.optimizers.schedules import PolynomialDecay\n\nbatch_size = 8\nnum_epochs = 3\n\nnum_train_steps = len(tf_train_dataset) * num_epochs\nlr_scheduler = PolynomialDecay(\n    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps\n)\nfrom tensorflow.keras.optimizers import Adam\n\nopt = Adam(learning_rate=lr_scheduler)\nmodel.compile(optimizer=opt)\n\n# Train in mixed-precision float16\ntf.keras.mixed_precision.set_global_policy("mixed_float16")\n'

In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

NameError: name 'train_dataloader' is not defined

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained('/kaggle/working/out', save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained('/kaggle/working/out')
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/18507 [00:00<?, ?it/s]

  0%|          | 0/686 [00:00<?, ?it/s]

## **Evaluation: Kinori**

In [29]:
my_model= '/kaggle/input/fft_nllb/pytorch/default/1'
source= 'hin_Deva'
target= 'kan_Deva'

In [30]:
fft_model= AutoModelForSeq2SeqLM.from_pretrained(my_model)

In [31]:
from transformers import pipeline
translator = pipeline('translation', model=fft_model, tokenizer=tokenizer, src_lang=source, tgt_lang=target)

In [34]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)
test_dataloader

In [35]:
# Acutal translation: हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक मानुष सौर्मींदैसा।
translator("हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ लोग विदक जाते हैं।")

[{'translation_text': 'हिन्दी-हिमाचली पहाड़ी भाषा रोह नाऔ शूनिंदैए दौक मानुष विदक नाशदैसा।'}]

In [39]:
"""
results

{'score': 15.669726436489944,
 'counts': [17796, 7979, 4025, 2089],
 'totals': [46021, 40538, 35080, 29727],
 'precisions': [38.66930314421677,
  19.682766786718634,
  11.473774230330672,
  7.027281595855619],
 'bp': 0.9955770512173886,
 'sys_len': 46021,
 'ref_len': 46225}
"""

"\nresults\n\n{'score': 15.669726436489944,\n 'counts': [17796, 7979, 4025, 2089],\n 'totals': [46021, 40538, 35080, 29727],\n 'precisions': [38.66930314421677,\n  19.682766786718634,\n  11.473774230330672,\n  7.027281595855619],\n 'bp': 0.9955770512173886,\n 'sys_len': 46021,\n 'ref_len': 46225}\n"

In [37]:
test_dataset

Dataset({
    features: ['Hindi', 'Kinori'],
    num_rows: 6093
})

In [38]:
sample_data = test_dataset.select(range(20))  

df = pd.DataFrame(sample_data)

translations = []
for index, row in df.iterrows():
    hindi_text = row['Hindi']
    kinori_target = row['Kinori']

    translation = translator(hindi_text, max_length=128)[0]['translation_text']

    translations.append({
        'Hindi': hindi_text,
        'Target Kinori': kinori_target,
        'Generated Kinori': translation
    })

results_df = pd.DataFrame(translations)
results_df

,Hindi,Target Kinori,Generated Kinori
0,कभी भी किसी कार्य में विघ्न डालने के लिए पंडित...,कौदु बि कुन काम कै मुश्कौल बेज़नाई ताईंयै पंडि...,कौदु बि कुन काम कै विघ्न बेज़नाई ताईंयै पंडित ...
1,हरगंगे! भई हरगंगे! \n,हरगंगे! भाई हरगंगे! \n,हरगंगे! भाई हरगंगे!
2,क्योंकि यह मिट्टी \n,किजु बोलताह होई माटिह\n,किजु बोल्ताह होई माटिह
3,यह जगह शहर में सबसे अच्छा बर्गर है! \n,हौयौ ज़ागा कैह शहर रोह सेबका औबौल बर्गरस। \n,होई ज़ागा शहर केह सऐब काह बोलोह बर्गर साह।
4,कितना बदल गया\n,केतिग बोदली।\n,केतिग बोदलुई
5,नई नई दलीलें अपीलें करके \n,नोऊवोंह-नोऊवोह बाते होरिह ज़ुर्राई देह।\n,नोईयोह नोईयोह दलील अपील टानिह देह
6,उसने झट से सुदामा की भेड़ चुराकर अपनी भेड़ों ...,तियूँवै फौटौक ज़ैई सुदामा रि बेरी च़ोरिंदै आपन...,तिनिये सोदामा रोह बेरी च़ोरिह देह आफरोह बेरी क...
7,हाथ जोड़ आते यहाँ लाखों ही सवाली\n,हाथ जोडाईदेह आछदेह साह ईनचे लाखों सवाली \n,हाथ जोडाईदेह आछ़तोह ईन्छ़े लाखो सवाली
8,या यूं कहें वे नीला आकाश न देख पाने को अभिशप्त...,या हैऔ बोलनौ तेनै नीला आकाश नेदेखी बोलबैलै दोश...,याह होयोह बोलेन थियेह सोह निलोह सोरोग नाई शाईम...
9,कुश्ती में पहने हुए उसके नए कपड़े लगाकर कुछ प...,कुश्ती कै लाउवैंदै तियूँरै नौउवै गाछ़ै गिनुविं...,कुश्ती कैह बेज़िह देह तिन रोह नोईयोह गाछ़ोह बे...


## **Loading and preparing the Dataset: Kangri**

In [52]:
hindi_path= '/kaggle/input/kangri-train/Kr_4_Hindi.txt'
kangri_path= '/kaggle/input/kangri-train/Kr_4_kangri.txt'

def r_and_f(file_path):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line:
                sentences.append(stripped_line)
    return sentences

def align_sentences(hindi_sen, kangri_sen):
    min_len = min(len(hindi_sen), len(kangri_sen))
    aligned_hindi = hindi_sen[:min_len]
    aligned_kangri = kangri_sen[:min_len]
    return aligned_hindi, aligned_kangri

hindi_sen = r_and_f(hindi_path)
kangri_sen = r_and_f(kangri_path)

hindi_sen, kangri_sen = align_sentences(hindi_sen, kangri_sen)

print(f"Number of aligned Hindi sentences: {len(hindi_sen)}")
print(f"Number of aligned Kangri sentences: {len(kangri_sen)}")

Number of aligned Hindi sentences: 26785
Number of aligned Kangri sentences: 26785


In [53]:
for i in range(5):
    print(f"Hindi: {hindi_sen[i]}")
    print(f"Kangri: {kangri_sen[i]}")
    print("--"*40)

Hindi: जब तक गैस नहीं थी, सोते समय मेरी आँखें दुःखती रहती थीं, धुंए से।
Kangri: जदैड़ी तक गैस नीं थी, सौंदीयां फेरा मेरीयां हाखीं दुखदीयां थीयां, धुयें नै।
--------------------------------------------------------------------------------
Hindi: बबलू और लीना भी आ गए।
Kangri: बबलू कन्नै लीना बी आई गये।
--------------------------------------------------------------------------------
Hindi: मैं उनको भी पूछती हूँ, कुछ खाना पीना होगा तो।
Kangri: मैं तीनां जो भी पुछदी, कुछ खाणा पीणा हुंगा ता।
--------------------------------------------------------------------------------
Hindi: कांगड़ी से हिन्दी रूपान्तरण - 6
Kangri: कांगडी - 6
--------------------------------------------------------------------------------
Hindi: मेरी तो इन वर्तनों को मांज-मांज कर बस हो गई ।
Kangri: मेरी ता इनां भांडेयां मांजी मांजी न होई गई।
--------------------------------------------------------------------------------


In [54]:
dataset_hk= list(zip(hindi_sen, kangri_sen))

import csv
import pandas as pd

file_name= "hi-kangri-translations.csv"

with open(file_name, 'w', encoding= 'utf-8') as file:
    writer= csv.writer(file)
    writer.writerow(['Hindi', 'Kangri'])
    
    for Hindi, Kangri in dataset_hk:
        writer.writerow([Hindi, Kangri])

In [55]:
dataframe_hk= pd.read_csv("/kaggle/working/hi-kangri-translations.csv")
dataframe_hk

,Hindi,Kangri
0,"जब तक गैस नहीं थी, सोते समय मेरी आँखें दुःखती ...","जदैड़ी तक गैस नीं थी, सौंदीयां फेरा मेरीयां हाख..."
1,बबलू और लीना भी आ गए।,बबलू कन्नै लीना बी आई गये।
2,"मैं उनको भी पूछती हूँ, कुछ खाना पीना होगा तो।","मैं तीनां जो भी पुछदी, कुछ खाणा पीणा हुंगा ता।"
3,कांगड़ी से हिन्दी रूपान्तरण - 6,कांगडी - 6
4,मेरी तो इन वर्तनों को मांज-मांज कर बस हो गई ।,मेरी ता इनां भांडेयां मांजी मांजी न होई गई।
...,...,...
26780,महीना आ गया था,"मैं तियार आं, करा मेरा ब्याह-, दिक्खा कोई कुड़ी"
26781,ठंड भी बहुत बढ़ रही थी,"एह ई रोज्जे-2 दी लड़ाई ऐ नां-"""
26782,और यही महीना शादियों से अभी भरा हुआ था,"गुस्से च जोरै घांऐ गलाया तिन्नी, करी चला गेया ..."
26783,सरपंच के साथ मिलकर श्याम वरयाम परमेश्वरी उषा औ...,"लाज्जो वी भड़कदी-2 उआंन्ने चीयां पेई, करी बाहर..."


In [56]:
repeat_times = 2
augmented_dataframe = pd.concat([dataframe_hk] * repeat_times)
augmented_dataframe = augmented_dataframe.reset_index(drop=True)
augmented_dataframe.to_csv("/kaggle/working/augmented_hi-kangri-translations.csv")
augmented_dataframe

,Hindi,Kangri
0,"जब तक गैस नहीं थी, सोते समय मेरी आँखें दुःखती ...","जदैड़ी तक गैस नीं थी, सौंदीयां फेरा मेरीयां हाख..."
1,बबलू और लीना भी आ गए।,बबलू कन्नै लीना बी आई गये।
2,"मैं उनको भी पूछती हूँ, कुछ खाना पीना होगा तो।","मैं तीनां जो भी पुछदी, कुछ खाणा पीणा हुंगा ता।"
3,कांगड़ी से हिन्दी रूपान्तरण - 6,कांगडी - 6
4,मेरी तो इन वर्तनों को मांज-मांज कर बस हो गई ।,मेरी ता इनां भांडेयां मांजी मांजी न होई गई।
...,...,...
53565,महीना आ गया था,"मैं तियार आं, करा मेरा ब्याह-, दिक्खा कोई कुड़ी"
53566,ठंड भी बहुत बढ़ रही थी,"एह ई रोज्जे-2 दी लड़ाई ऐ नां-"""
53567,और यही महीना शादियों से अभी भरा हुआ था,"गुस्से च जोरै घांऐ गलाया तिन्नी, करी चला गेया ..."
53568,सरपंच के साथ मिलकर श्याम वरयाम परमेश्वरी उषा औ...,"लाज्जो वी भड़कदी-2 उआंन्ने चीयां पेई, करी बाहर..."


In [57]:
from sklearn.model_selection import train_test_split
train_dataset, evaluation_dataset= train_test_split(augmented_dataframe, test_size=0.1, random_state=40)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)

Training dataset shape:  (48213, 2)
Validation dataset shape:  (5357, 2)


In [58]:
from datasets import Dataset

train_dataset= Dataset.from_pandas(train_dataset)
val_dataset= Dataset.from_pandas(evaluation_dataset)

train_dataset= train_dataset.remove_columns("__index_level_0__")
val_dataset= val_dataset.remove_columns("__index_level_0__")

In [59]:
import datasets
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
})

In [60]:
main_dataset

DatasetDict({
    train: Dataset({
        features: ['Hindi', 'Kangri'],
        num_rows: 48213
    })
    validation: Dataset({
        features: ['Hindi', 'Kangri'],
        num_rows: 5357
    })
})

## **Tokenizing and getting ready for Training: Kangri**

In [15]:
from tqdm.auto import tqdm, trange
import random

texts_with_unk = [
    text for text in tqdm(dataframe_hk.Hindi) 
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))

  0%|          | 0/26785 [00:00<?, ?it/s]

318


In [16]:
texts_with_unk = [
    text for text in tqdm(dataframe_hk.Kangri) 
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))

  0%|          | 0/26785 [00:00<?, ?it/s]

648


In [8]:
from transformers import NllbTokenizer
s_lang="hin_Deva"
t_lang="kangri_Deva"
tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M', src_lang=s_lang, tgt_lang=t_lang)

In [9]:
def fix_tokenizer(tokenizer, new_lang='kangri_Deva'):

    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang

    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}
    
    return tokenizer

tokenizer= fix_tokenizer(tokenizer)

In [10]:
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(256205, 1024)

In [11]:
added_token_id = tokenizer.convert_tokens_to_ids('kangri_Deva')
similar_lang_id = tokenizer.convert_tokens_to_ids('hin_Deva')
embeds = model.model.shared.weight.data
# moving the embedding for "mask" to its new position
embeds[added_token_id+1] =embeds[added_token_id]
# initializing new language token with a token of a similar language
embeds[added_token_id] = embeds[similar_lang_id]

In [61]:
max_length= 128

def preprocess_function(examples):
    inputs= [ex for ex in examples["Hindi"]]
    targets= [ex for ex in examples["Kangri"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [62]:
tokenized_datasets = main_dataset.map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/48213 [00:00<?, ? examples/s]

Map:   0%|          | 0/5357 [00:00<?, ? examples/s]

In [63]:
tokenized_datasets= tokenized_datasets.remove_columns(['Hindi', 'Kangri'])

In [64]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 48213
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5357
    })
})

In [65]:
from transformers import DataCollatorForSeq2Seq
#generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt", pad_to_multiple_of=128)

#Pytorch
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [66]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

## **Training the Model: Kangri**

In [32]:
import evaluate
metric = evaluate.load("sacrebleu")

In [70]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [69]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [68]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained('/kaggle/working/out', save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained('/kaggle/working/out')
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/18081 [00:00<?, ?it/s]

  0%|          | 0/670 [00:00<?, ?it/s]

In [75]:
"""
results
{'score': 0.09152850478601893,
 'counts': [1192, 74, 19, 3],
 'totals': [55718, 50361, 45044, 39894],
 'precisions': [2.139344556516745,
  0.1469390997001648,
  0.04218097859870349,
  0.007519927808693036],
 'bp': 0.9159447113479372,
 'sys_len': 55718,
 'ref_len': 60610}
 """

"\nresults\n{'score': 0.09152850478601893,\n 'counts': [1192, 74, 19, 3],\n 'totals': [55718, 50361, 45044, 39894],\n 'precisions': [2.139344556516745,\n  0.1469390997001648,\n  0.04218097859870349,\n  0.007519927808693036],\n 'bp': 0.9159447113479372,\n 'sys_len': 55718,\n 'ref_len': 60610}\n "

## **Evaluation: Kangri**

In [20]:
my_model_hk= '/kaggle/input/nllb_ft_kangri/pytorch/default/1'
source= 'hin_Deva'
target= 'kangri_Deva'

In [21]:
fft_model_hk= AutoModelForSeq2SeqLM.from_pretrained(my_model_hk)

In [22]:
from transformers import pipeline
translator = pipeline('translation', model=fft_model_hk, tokenizer=tokenizer, src_lang=source, tgt_lang=target)

In [23]:
hindi_path= '/kaggle/input/kangri-dataset/Kr_4_Hindi_test.txt'
kangri_path= '/kaggle/input/kangri-dataset/Kr_4_Kangri_test.txt'

def r_and_f(file_path):
    sentences= []
    with open(file_path, 'r', encoding= 'utf-8') as file:
        for line in file:
            stripped_line= line.strip()
            if stripped_line:
                sentences.append(line)
                
    return sentences

hindi_sen= r_and_f(hindi_path)
kangri_sen= r_and_f(kangri_path)

if len(hindi_sen) != len(kangri_sen):
    raise ValueError("The number of sentences do not match!")
else:
    print("The number of sentences match!") 

The number of sentences match!


In [25]:
dataset_hk_test= list(zip(hindi_sen, kangri_sen))

import csv
file_name= "hi-kangri-test-translations.csv"

with open(file_name, 'w', encoding= 'utf-8') as file:
    writer= csv.writer(file)
    writer.writerow(['Hindi', 'Kangri'])
    
    for Hindi, Kangri in dataset_hk_test:
        writer.writerow([Hindi, Kangri])

In [30]:
import pandas as pd
from datasets import Dataset
test_dataframe= pd.read_csv("hi-kangri-test-translations.csv")
test_dataset= Dataset.from_pandas(test_dataframe)

In [31]:
test_dataset

Dataset({
    features: ['Hindi', 'Kangri'],
    num_rows: 500
})

In [33]:
sample_data = test_dataset.select(range(20))  

df = pd.DataFrame(sample_data)

translations = []
for index, row in df.iterrows():
    hindi_text = row['Hindi']
    kangri_target = row['Kangri']

    translation = translator(hindi_text, max_length=128)[0]['translation_text']

    translations.append({
        'Hindi': hindi_text,
        'Target Kangri': kangri_target,
        'Generated Kangri': translation
    })

results_df = pd.DataFrame(translations)
results_df

,Hindi,Target Kangri,Generated Kangri
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ...,हिमाचली - म्हाचली पहाड़ी भासा दा ना सुणी के ई ...,हिन्दी-हिमाचली पहाड़ी भाषा दा नां सुणी करी ही ...
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों क...,जे इस मुद्दे पर दूए परदेसां दे बसिंदेयां जो को...,जे इस मुसले पर दूएं प्रदेशां दे निवासीयां जो क...
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की...,अपर अपणे ई परदेसे दी भासा ताईं लोकां दे दी घं...,पर अपणे ही प्रदेश दी भाषा दी तरफ़ लोकां दे मने...
3,पता नहीं ऐसा कर के क्या हासिल‌ होगा ।\n,पता नी देह्या करी के क्या हासल होणा ।\n,पता नी ऐसा करी के क्या हासिल हुंगा
4,अगर हम अपनी माता ( हिमाचली भाषा ) के नहीं हो स...,जे अहां अपणियां माऊ दे नी होई सकदे तां मतेइया ...,जे असां अपणी माता (महाचली भाषा) दे नी हो सकदे ...
5,ऐसा तो है नहीं कि हिमाचली पहाड़ी का पुराना अस्...,देह्या तां है नी भई म्हाचली पहाड़ी भासा दा परा...,एह ता है नी कि म्हाचली पहाड़ी दा पुराना अस्तित...
6,हिमाचली पहाड़ी भाषा के साथ यहां के स्थानीय लोग...,म्हाचली पहाड़ी भासा सौग्गी इत्थु दे थान्नी ( स...,हिमाचल पहाड़ी भाषा कने इत्थू दे स्थानीय लोकां ...
7,हालांकि सन 920 ई से पहले का साहित्य लिखित रूप...,ल्हांकि सन 920 ई ते पैहल्का (साहित )साहित्य लि...,हालांकि सन 920 ई ते पैले दा साहित्य लिखित रूपे...
8,जो कुछ मिलता भी है वो लोकगीतों और भजनों के रूप...,जे किच्छ मिलदा बि ऐ सैह लोकगीतां कनै भजना दे र...,जे कुछ मिलदा बी सैह लोकगीतां कने भजनों दे रूपे...
9,"शिलालेख / राजाओं के लिखे पत्र /संधिपत्र , चर्प...",शिलालेख/ राजेयां दियां लिखियां चिट्ठियां / समझ...,"शिलालेख / राजेयां दे लिखे पत्र /संधिपत्र , चर्..."
